In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes

# library to handle HTTP requests
import requests 
# for xml & html scrapping 
from bs4 import BeautifulSoup

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

import requests # library to handle requests
import json # library to handle JSON files

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> conda-forge::ca-certificates-2020.12.5-ha878542_0
  certifi                anaconda::certifi-2020.6.20-py36_0 --> conda-forge::certifi-2020.12.5-py36h5fab9bb_0
  openssl               anaconda::openssl-1.1.1h-h7b6447c_0 --> conda-forge::openssl-1.1.1i-h7f98852_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!conda install -c anaconda beautifulsoup4 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.12.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2020.12.5-py36h5~ --> anaconda::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1i-h7f98852_0 --> anaconda::openssl-1.1.1h-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!pip install lxml

### Scraping the Wikipedia page for the table of postal codes of Canada

In [4]:
#get url
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

#Display title
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

### Convert html table to Pandas DataFrame for cleaning and preprocessing.

In [5]:
# extracting the raw table inside that webpage
tabl = soup.find("table")

# get columns
tabl.tr.text

table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Data preprocessing and cleaning
**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [6]:
# Get names of indexes for which column Borough has value "Not assigned"
df_borough = df[df['Borough'] == 'Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(df_borough, inplace=True)
df.reset_index(drop= True,inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Creating dataframe with coordinates
**Import the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada**

In [7]:
df_la_lo = pd.read_csv('https://cocl.us/Geospatial_data')
df_la_lo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge the coordinates and the list of postal codes of Canada on 'Postal Code'.**

In [8]:
#merge the two dataframes
df = pd.merge(df,df_la_lo,on='Postal Code')
df.shape

(103, 5)

### Explore the neighbourhoods of Canada which contain Toronto in their Borough

In [9]:
df_toronto = df[df['Borough'].str.contains('Toronto',regex=False)]
display(df_toronto.shape)
df_toronto.reset_index(drop=True)
df_toronto.head(10)

(39, 5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


### Visualize all the Neighbourhoods in the city of Toronto using Folium

In [10]:
#Latitude and longitude coordinates are: 43.651070, -79.347015

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

### Cluster Neighborhoods

**Run _k_-means to cluster the neighborhood into 5 clusters.**

In [11]:
# set number of clusters
k=5

#create cluster dataframe
toronto_clustering = df_toronto.drop(['Postal Code','Borough','Neighbourhood'],1)

# run k-means clusteringz
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_

# add clustering labels
df_toronto.insert(5, 'Cluster Labels', kmeans.labels_)


In [12]:
df_toronto.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1


**Visualize the resulting clusters**

In [13]:
# create map
latitude = 43.651070
longitude=-79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters for K = 5

In [14]:
# cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1, 0, 3, 4, 0, 3, 4, 0, 4, 2, 2, 2, 2,
       1, 2, 3, 1, 2, 3, 1, 2, 3, 2, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

In [15]:
# This will create a dataframe with borough of each neighborhood which we will merge with each cluster dataframe
# df_cluster = df_toronto.copy(deep=True)
df_cluster = df_toronto[['Borough', 'Neighbourhood','Cluster Labels']]

df_cluster.head(10)

,Borough,Neighbourhood,Cluster Labels
2,Downtown Toronto,"Regent Park, Harbourfront",0
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0
9,Downtown Toronto,"Garden District, Ryerson",0
15,Downtown Toronto,St. James Town,0
19,East Toronto,The Beaches,4
20,Downtown Toronto,Berczy Park,0
24,Downtown Toronto,Central Bay Street,0
25,Downtown Toronto,Christie,3
30,Downtown Toronto,"Richmond, Adelaide, King",0
31,West Toronto,"Dufferin, Dovercourt Village",1


#### How many Neighborhoods per Cluster?

In [16]:
# How many neighborhoods in each cluster
df_cluster['Cluster Labels'].value_counts()

0    16
2     8
3     6
4     5
1     4
Name: Cluster Labels, dtype: int64

In [17]:
cluster1 = df_cluster.loc[df_cluster['Cluster Labels'] == 0]
cluster1

,Borough,Neighbourhood,Cluster Labels
2,Downtown Toronto,"Regent Park, Harbourfront",0
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0
9,Downtown Toronto,"Garden District, Ryerson",0
15,Downtown Toronto,St. James Town,0
20,Downtown Toronto,Berczy Park,0
24,Downtown Toronto,Central Bay Street,0
30,Downtown Toronto,"Richmond, Adelaide, King",0
36,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",0
42,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",0
48,Downtown Toronto,"Commerce Court, Victoria Hotel",0


In [18]:
cluster2 = df_cluster.loc[df_cluster['Cluster Labels'] == 1]
cluster2

,Borough,Neighbourhood,Cluster Labels
31,West Toronto,"Dufferin, Dovercourt Village",1
69,West Toronto,"High Park, The Junction South",1
75,West Toronto,"Parkdale, Roncesvalles",1
81,West Toronto,"Runnymede, Swansea",1


In [19]:
cluster3 = df_cluster.loc[df_cluster['Cluster Labels'] == 2]
cluster3

,Borough,Neighbourhood,Cluster Labels
61,Central Toronto,Lawrence Park,2
62,Central Toronto,Roselawn,2
67,Central Toronto,Davisville North,2
68,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",2
73,Central Toronto,"North Toronto West, Lawrence Park",2
79,Central Toronto,Davisville,2
83,Central Toronto,"Moore Park, Summerhill East",2
86,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",2


In [20]:
cluster4 = df_cluster.loc[df_cluster['Cluster Labels'] == 3]
cluster4

,Borough,Neighbourhood,Cluster Labels
25,Downtown Toronto,Christie,3
37,West Toronto,"Little Portugal, Trinity",3
43,West Toronto,"Brockton, Parkdale Village, Exhibition Place",3
74,Central Toronto,"The Annex, North Midtown, Yorkville",3
80,Downtown Toronto,"University of Toronto, Harbord",3
84,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",3


In [21]:
cluster5 = df_cluster.loc[df_cluster['Cluster Labels'] == 4]
cluster5

,Borough,Neighbourhood,Cluster Labels
19,East Toronto,The Beaches,4
41,East Toronto,"The Danforth West, Riverdale",4
47,East Toronto,"India Bazaar, The Beaches West",4
54,East Toronto,Studio District,4
100,East Toronto,"Business reply mail Processing Centre, South C...",4
